In [24]:
import pandas as pd
import pickle

In [3]:
df = pd.read_csv("../isec_train_scaled.csv", index_col=0)
df
X, y = df.iloc[:, :-1], df.iloc[:, -1]

In [15]:
print(y.iloc[0])

0.0


In [5]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=4, random_state = 32)

In [9]:
def train(model, X, y):
	# 모델 학습
	model.fit(X, y)

In [7]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
def get_clf_eval(y, y_pred, vervose=True):
	confusion = confusion_matrix(y, y_pred)
	accuracy = accuracy_score(y, y_pred)
	precision = precision_score(y, y_pred)
	recall = recall_score(y, y_pred)
	F1 = f1_score(y, y_pred)
	AUC = roc_auc_score(y, y_pred)
	if vervose:
		print('오차행렬:\n', confusion)
		print('\n정확도: {:.4f}'.format(accuracy))
		print('정밀도: {:.4f}'.format(precision))
		print('재현율: {:.4f}'.format(recall))
		print('F1: {:.4f}'.format(F1))
		print('AUC: {:.4f}'.format(AUC))
	return confusion, accuracy, precision, recall, F1, AUC


# hold out

In [11]:
# scikit-learn 패키지의 train_test_split()을 불러옵니다.
from sklearn.model_selection import train_test_split
# 학습 데이터와 학습용과 검증용으로 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(X, y,
													test_size=0.2,
													random_state=42)
train(model, X_train, y_train)
y_pred = model.predict(X_test)
get_clf_eval(y_test, y_pred)

오차행렬:
 [[46  0]
 [ 0 54]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000


(array([[46,  0],
        [ 0, 54]]),
 1.0,
 np.float64(1.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(1.0))

cross validation

In [25]:
from sklearn.model_selection import KFold
knum = 10
kfold = KFold(n_splits=knum, shuffle=True, random_state=42)
model = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=4, random_state = 32)
max_acc, max_f1, max_auc = 0, 0, 0
for train_index, test_index in kfold.split(X):
	X_train, X_test = X.iloc[train_index], X.iloc[test_index]
	y_train, y_test = y.iloc[train_index], y.iloc[test_index]
	# 학습 및 예측
	train(model, X_train, y_train)
	y_pred = model.predict(X_test)
	_, accuracy, _, _, F1, AUC = get_clf_eval(y_test, y_pred)
	if max_acc < accuracy:
		max_acc = accuracy
		with open('models/max_acc.pickle','wb') as fw:
			pickle.dump(model, fw)
	if max_f1 < F1:
		max_f1 = F1
		with open('models/max_f1.pickle','wb') as fw:
			pickle.dump(model, fw)
	if max_auc < AUC:
		max_auc = AUC
		with open('models/max_auc.pickle','wb') as fw:
			pickle.dump(model, fw)


오차행렬:
 [[23  0]
 [ 0 27]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[23  0]
 [ 0 27]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[24  0]
 [ 0 26]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[25  0]
 [ 0 25]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[30  0]
 [ 0 20]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[22  0]
 [ 0 28]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[30  0]
 [ 0 20]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[23  0]
 [ 0 27]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[23  0]
 [ 0 27]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[27  0]
 [ 0 23]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000


In [26]:
from sklearn.model_selection import StratifiedKFold
knum = 10
model = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=4, random_state = 32)
skf = StratifiedKFold(n_splits=knum)
max_acc, max_f1, max_auc = 0, 0, 0
for train_index, test_index in skf.split(X, y):
	X_train, X_test = X.iloc[train_index], X.iloc[test_index]
	y_train, y_test = y.iloc[train_index], y.iloc[test_index]
	train(model, X_train, y_train)
	y_pred = model.predict(X_test)
	_, accuracy, _, _, F1, AUC = get_clf_eval(y_test, y_pred)
	if max_acc < accuracy:
		max_acc = accuracy
		with open('models/max_acc.pickle','wb') as fw:
			pickle.dump(model, fw)
	if max_f1 < F1:
		max_f1 = F1
		with open('models/max_f1.pickle','wb') as fw:
			pickle.dump(model, fw)
	if max_auc < AUC:
		max_auc = AUC
		with open('models/max_auc.pickle','wb') as fw:
			pickle.dump(model, fw)


오차행렬:
 [[25  0]
 [ 0 25]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[25  0]
 [ 0 25]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[25  0]
 [ 0 25]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[25  0]
 [ 0 25]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[25  0]
 [ 0 25]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[25  0]
 [ 0 25]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[25  0]
 [ 0 25]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[25  0]
 [ 0 25]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[25  0]
 [ 0 25]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[25  0]
 [ 0 25]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
